## 简介
爬取京东某本书的评论，然后进行分析，选择的书为：《ES6标准入门（第3版）》 、阮一峰 著 、博文视点出品、（电子工业出版社 书店）。
该书有2.1万评论，其它书店该书评论寥寥无几，进行数据分析一探究竟。

地址：https://item.jd.com/12172449.html

In [ ]:
```
    唯一变量为page
```
# page_num = 0
# url = "https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=12172449&score=0&sortType=5&page="+page_num+
# "&pageSize=10&isShadowSku=0&rid=0&fold=0"

## 爬取评论&数据清洗

In [1]:
import requests
import re
import pandas as pd
import os

In [2]:
headers1 = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    # 'cookie':  'UM_distinctid=172b674e5f71cb-034c5dfe424061-3a365305-1fa400-172b674e5f8848; PHPSESSID=4gsduu0v6tmi1lkbsf8cp7kej0; local_zone=320100%7Cnanjing%7C%C4%CF%BE%A9%7C58238; Hm_lvt_d64174522c86449826babe56fb2a88ff=1592199015; CNZZDATA1089612=cnzz_eid%3D725595362-1592200519-https%253A%252F%252Fguilin.cncn.com%252F%26ntime%3D1592200519; Hm_lpvt_d64174522c86449826babe56fb2a88ff=1592203476',
    'pragma': 'no-cache',
    # 'referer':  'https://beihai.cncn.com/jingdian/beihai/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',

}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    # 'cookie':  'UM_distinctid=172b674e5f71cb-034c5dfe424061-3a365305-1fa400-172b674e5f8848; PHPSESSID=4gsduu0v6tmi1lkbsf8cp7kej0; local_zone=320100%7Cnanjing%7C%C4%CF%BE%A9%7C58238; Hm_lvt_d64174522c86449826babe56fb2a88ff=1592199015; CNZZDATA1089612=cnzz_eid%3D725595362-1592200519-https%253A%252F%252Fguilin.cncn.com%252F%26ntime%3D1592200519; Hm_lpvt_d64174522c86449826babe56fb2a88ff=1592203476',
    'pragma': 'no-cache',
    # 'referer':  'https://beihai.cncn.com/jingdian/beihai/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',

}


def run():
    ```
    正则表达式匹配串
    content: 评论
    score: 星级
    creationTime: 评论时间
    ```
    content = re.compile(r"content\":\"\S*\",\"creationTime")
    score = re.compile("score\":[0-5]")
    creationTime = re.compile("[0-9]*-[0-9]*-[0-9]*\s[0-9]*:[0-9]*:[0-9]*")
    
    
    ```
    总共有 100页，只提取主评论（追加评论不提，因为没有star）
    每页10个评论 
    官网显示总共2.1w评论，1.6w默认
    提取数据为652条
    ```
    for i in range(0,99):
        page_num = str(i)
        q_url = "https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=12172449&score=0&sortType=5&page="+page_num+"&pageSize=10&isShadowSku=0&rid=0&fold=0"

        # 获取评论json数据
        url_first = q_url
        html = requests.get(url_first, headers1)

        # 获取基本数据集
        text = []
        star = []
        time = []
        text = content.findall(html.text)
        star = score.findall(html.text)
        time = creationTime.findall(html.text)

        # 数据清理
        star_ = []
        time_ = []
        text_ = []
        for k in range(0,len(text)):
            star_.append(star[k][-1:])
            time_.append(time[k])
            text_.append(text[k][10:-15])

        # 保存为csv
        data = {'star':star_,'time':time_,'text':text_}
        file_name = 'data.csv'
        df = pd.DataFrame(data)
        if os.path.exists(file_name):
            df.to_csv(file_name, mode='a', index=False, na_rep='NA',header=False, encoding='utf-8')
        else:
            df.to_csv(file_name, mode='a', index=False, na_rep='NA',header=['score', 'creationTime', 'content'], encoding='utf-8')

if __name__ == '__main__':
    data = run()

## 数据分析
使用 jieba 分词对一个文本进行分词，统计次数出现最多的词语

In [ ]:
import jieba

txt = open("data.csv", "r", encoding='utf-8').read()
words = jieba.lcut(txt)     # 使用精确模式对文本进行分词
counts = {}     # 通过键值对的形式存储词语及其出现的次数

for word in words:
    if len(word) == 1:    # 单个词语不计算在内
        continue
    else:
        counts[word] = counts.get(word, 0) + 1    # 遍历所有词语，每出现一次其对应的值加 1

items = list(counts.items())
items.sort(key=lambda x: x[1], reverse=True)    # 根据词语出现的次数进行从大到小排序

for i in range(3):
    word, count = items[i]
    print("{0:<5}{1:>5}".format(word, count))

## 测试 [LAC](https://github.com/baidu/lac)百度中文词法分析（分词+词性+专名）系统

In [1]:
from LAC import LAC

# 装载分词模型
lac = LAC(mode='seg')

# 单个样本输入，输入为Unicode编码的字符串
text = u"LAC是个优秀的分词工具"
seg_result = lac.run(text)

# 批量样本输入, 输入为多个句子组成的list，平均速率会更快
texts = [u"LAC是个优秀的分词工具", u"百度是一家高科技公司"]
seg_result = lac.run(texts)

ModuleNotFoundError: No module named 'LAC'